In [3]:
import pandas as pd
import numpy as np
import networkx as nx

In [4]:
interaction_df = pd.read_csv("interaction_data.csv")
filtered_df = pd.read_csv('filtered_data.csv')
result_df = pd.read_csv('result_data.csv')
protein_pairs_df = pd.read_csv('protein_pairs_data.csv')
graph = nx.read_graphml("graph_file.graphml")


In [34]:
def create_subgraph(G, subgraph_nodes):
    return nx.induced_subgraph(G, subgraph_nodes)

def get_node_list(graph):
    return list(graph.nodes())

def get_neighbor_list(graph, node):
    return list(graph.neighbors(node))

def get_degree_neighbor_tuple_list(graph, node):
    neighbor_list = list(graph.neighbors(node))
    neighbor_list_with_degree = [(graph.degree(node), node) for node in neighbor_list]
    return neighbor_list_with_degree

def get_degree_map(graph, node_list):
    degree_map = {}
    for node in node_list:
        degree_map[node] = graph.degree(node) 
    return degree_map

def calculate_cc(G, node_list):
    subG = create_subgraph(G, node_list)
    E = subG.number_of_edges()
    V = subG.number_of_nodes()
    cc = 0
    try:
        cc = 2 * E / (V * (V - 1))
    except ZeroDivisionError:
        pass
    return cc

def calculate_NA(node_list1, node_list2):
    common_elements = 0
    for ele1 in node_list1:
        if ele1 in node_list2: common_elements += 1
    
    if len(node_list1) == 0 or len(node_list2) == 0:
        return 0

    return common_elements*common_elements/(len(node_list1) * len(node_list2))

def LCMA1(graph):
    LC = set()
    adj_list = {}
    node_list = get_node_list(graph)
    for node in node_list:
        adj_list[node] = get_degree_neighbor_tuple_list(graph, node)  # unsorted
    
    count = 0
    for node in node_list:
        node_subset_list = [i for _, i in adj_list[node]]
        node_subset_list.append(node)
        cc = calculate_cc(graph, node_subset_list)
        degree_map = get_degree_map(graph, node_subset_list)
        stop = False
        while not stop:
            cc_new = 0
            try:
                _, new_node = min(adj_list[node])
                new_node_subset_list = node_subset_list.remove(new_node)
                cc_new = calculate_cc(graph, new_node_subset_list)
            except:
                pass
            if cc_new > cc:
                adj_list[node].remove((degree_map[new_node], new_node))
                for neighbor_node in adj_list[new_node]:
                    degree_map[neighbor_node] -= 1
                cc = cc_new
            else:
                final_node_subset_list = [i for _, i in adj_list[node]]
                LC.add((cc, tuple(final_node_subset_list)))
                count += 1
                if count % 100 == 1: print(f"Done {count}/ {len(node_list)}")
                
                stop = True
                
    for cc, node_list in LC.copy():
        if len(node_list) <= 2:
            LC.remove((cc,node_list))

    
    return LC

def LCMA2(LC):
    stop = False
    sum_density = 0
    for cc, _ in LC:
        sum_density += cc
    if len(LC) == 0:
        average_density = 0
    else:
        average_density = sum_density / len(LC)

    while not stop:
        C = set()
        unused = set() ## what is meant by unused set
        for _, node_list in LC:
            unused.add(tuple(node_list))
        for _, node_list1 in LC:
            S = set()
            S.add(node_list1)
            for _, node_list2 in LC:
                if node_list1 == node_list2:
                    continue
                NA = calculate_NA(node_list1, node_list2)
                if NA > average_density:  # omega??
                    print(f"{NA} and {average_density}")
                    S.add(tuple(node_list2))
                    if node_list2 in unused:
                        unused.remove(node_list2)
                    if node_list1 in unused:
                        unused.remove(node_list1)
                    print(f"Current size of S: {len(S)}")

            C = C.union(S)

        for node_list in unused:
            C = C.union(set(node_list))
        
        # Track progress
        print(f"Current size of C: {len(C)}")
        
        sum_cc = 0
        LC_new = set()
        for node_list in C:
            cc_value = calculate_cc(graph, node_list)
            sum_cc += cc_value
            LC_new.add((cc_value, node_list))
        if len(C) == 0:
            average_cc = 0
        else:
            average_cc = sum_cc / len(C)

        print(f"{average_cc} and {average_density} and {0.95 * average_density}")
        if average_cc > 0.95 * average_density:
            average_density = average_cc
            if len(LC) == len(LC_new):
                stop = True
            LC = LC_new
        else:
            stop = True
        
        # Track progress
        print(f"Current size of LC: {len(LC)}")
        print("--------------------------------------------")

    return LC

            
            
                
                
    
    

In [35]:
LC = LCMA1(graph)
print(len(LC))

Done 1/ 4379
Done 101/ 4379
Done 201/ 4379
Done 301/ 4379
Done 401/ 4379
Done 501/ 4379
Done 601/ 4379
Done 701/ 4379
Done 801/ 4379
Done 901/ 4379
Done 1001/ 4379
Done 1101/ 4379
Done 1201/ 4379
Done 1301/ 4379
Done 1401/ 4379
Done 1501/ 4379
Done 1601/ 4379
Done 1701/ 4379
Done 1801/ 4379
Done 1901/ 4379
Done 2001/ 4379
Done 2101/ 4379
Done 2201/ 4379
Done 2301/ 4379
Done 2401/ 4379
Done 2501/ 4379
Done 2601/ 4379
Done 2701/ 4379
Done 2801/ 4379
Done 2901/ 4379
Done 3001/ 4379
Done 3101/ 4379
Done 3201/ 4379
Done 3301/ 4379
Done 3401/ 4379
Done 3501/ 4379
Done 3601/ 4379
Done 3701/ 4379
Done 3801/ 4379
Done 3901/ 4379
Done 4001/ 4379
Done 4101/ 4379
Done 4201/ 4379
Done 4301/ 4379
1513


In [36]:
LC_final = LCMA2(LC)

0.5 and 0.46493761681092804
Current size of S: 2
0.5714285714285714 and 0.46493761681092804
Current size of S: 2
0.8 and 0.46493761681092804
Current size of S: 3
0.6666666666666666 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 3
0.5 and 0.46493761681092804
Current size of S: 2
0.6666666666666666 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 2
0.5 and 0.46493761681092804
Current size of S: 2
0.75 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 3
0.5714285714285714 and 0.46493761681092804
Current size of S: 4
0.5625 and 0.46493761681092804
Current size of S: 2
0.5625 and 0.46493761681092804
Current size of S: 3
0.5625 and 0.46493761681092804